In [ ]:
!ls -l

In [2]:
#!lscpu
!free -m

              total        used        free      shared  buff/cache   available
Mem:          15885        1011       12857         152        2015       14402
Swap:             0           0           0


In [ ]:
! rm cygnus_lib.*; rm mylib.*
! wget https://raw.githubusercontent.com/gmazzitelli/cygno/master/cygnus_lib.py
! wget https://raw.githubusercontent.com/gmazzitelli/cygno/master/mylib.py
import cygnus_lib as cy
import matplotlib.pyplot as plt
import numpy as np    
import ROOT
from root_numpy import hist2array
import mylib as my


In [ ]:
############################################################
#load pedestal run to be used
############################################################
#import scipy.stats as stats
#import numpy as np
import time
#from astropy.stats import sigma_clip, mad_std


# 2054-...  (Orca Flash cooled)     h5Bool = 0
# 3593-3596 (Orca Flash not cooled) h5Bool = 0
# 2357-2359 (Orca Fusion)           h5Bool = 1
# 3511-3521 (BSI Prime)             h5Bool = 2

h5Bool = 0
fromDriveBool = 0

if fromDriveBool==1:
  h5Bool = 0
  print ('##############')
  from google.colab import drive
  drive.mount('/content/gdrive')
  #!cp '/content/gdrive/My Drive/Colab Notebooks/h5files/histograms_Run02356.root' ./
  !cp '/content/gdrive/My Drive/Colab Notebooks/h5files/histograms_Run02358.root' ./  
  f = ROOT.TFile("histograms_Run02358.root")
  print ('Find Keys: '+str(len(f.GetListOfKeys())))
  pic = []
  for i,e in enumerate(f.GetListOfKeys()):
    #print e.GetName()
    pic.append(e.GetName())
  #print len(pic)



  #pic, wfm = cy.root_TH2_name(f)
  max_image = len(pic)
  max_wfm = len(wfm)
  print ("# of Images (TH2) Files: %d " % (max_image))
  print ("# of Waveform (TH2) Files: %d " % (max_wfm))
  N = len(pic)
  #N = 200    
elif h5Bool==1:
  dataSelection = "TES"
  runI = [2358] #2357
  iImg = cy.swift_listdir(cy.file2FullPathCygnus(dataSelection, runI[0], 'H5'))  
  N = len(iImg)
elif h5Bool==2:
  start_image_to_read = 0
  max_image_to_read = 100
  tag = 'TES'
  #####runI = [3520]
  #runI = [3514]
  runI = [2355]
  dirname = tag+"/Run"+str(runI[0])
  path = "https://swift.cloud.infn.it:8080/v1/AUTH_1e60fe39fba04701aa5ffc0b97871ed8/Cygnus/"
  iImg = cy.swift_listdir(dirname)[start_image_to_read+1:max_image_to_read+1]
  N = len(iImg)
else:
  tag = "LAB"
  runI = [2054]
  #runI = [2155] 
  #f = cy.swift_read_root_file(cy.swift_root_file(tag, run))
  f = cy.swift_read_root_file(cy.swift_root_file(tag, runI[0]))
  try:
    print ('Find Keys: '+str(len(f.GetListOfKeys())))
    pic, wfm = cy.root_TH2_name(f)
    del wfm
    max_image = len(pic)
    #max_wfm = len(wfm)
    print ("# of Images (TH2) Files: %d " % (max_image))
    #print ("# of Waveform (TH2) Files: %d " % (max_wfm))
    N = len(pic)
  except:
    print("File not found")
    
    
t0 = time.time()
N=100
Njump = 5 #ignore first images (usually they are unstable)
Nx = int(2048/1)
Ny = int(2048/1)

img_real = np.zeros((N-Njump, Nx, Ny), dtype=np.uint16)
print('>>> '+str(Nx))
Evts = Nx*Ny
pixel = np.zeros((2, Evts), dtype=np.uint16)
pixel = np.array(np.meshgrid(np.arange(0,Nx,dtype=np.uint16), np.arange(0,Ny,dtype=np.uint16))).T.reshape(-1,2).T.tolist()

# pixel = []
# for pixx in range(0, Nx):
#   for pixx in range(0, Ny):
#     pixel.append([pixx,pixy])
# pixel = np.array(pixel).T.tolist()



#test1 = []
k = -1
for iTr in range(Njump,N):
  k = k+1
  if iTr % 50 == 0:
    print (iTr)
  #if fromDriveBool:
  #  print 'test'
  if h5Bool == 1:
    #if (len(filter (lambda x : x == iTr, notFound)) > 0):
    #  continue
    filename = iImg[iTr]
    msg = downloadH5(filename,directory)
    if msg:
      print (filename)
    else:
      print("File not found")
      continue
    tmp_file = '/content'+filename.split('H5')[1]
    image = cy.read_image_h5(tmp_file)
    #test1.append(image[pixel])
  elif h5Bool == 2:
    try:
      filename = iImg[iTr]
      tmp_file = cy.swift_download_file(path+filename)
      image = io.imread(tmp_file)  # IMAGE
      #test1.append(image[pixel])
      img_real[k] = image[pixel]
    except:
      print("File %s not found" % filename)
  else:
    #using uint8 due to RAM memory...then all values > 255, will be set to 255
    #img_real[k, np.array(pixel)[0,:], np.array(pixel)[1,:]] = hist2array(f.Get(pic[iTr]))[pixel][hist2array(f.Get(pic[iTr]))[pixel]>255]=255
    img_real[k, np.array(pixel)[0,:], np.array(pixel)[1,:]] = hist2array(f.Get(pic[iTr]))[pixel]
    
#del pixel[:] #free RAM space
t1 = time.time()-t0  
print(t1)


In [ ]:
np.shape(pixel)

In [ ]:
############################################################
#functions used to simulate noise
############################################################

# created a ECDF (x,y) data
def ecdf(data):
  """ Compute ECDF """
  unique, counts = np.unique(data, return_counts=True)
  y = np.cumsum(counts)
  y = y / float(y[-1])
  y = np.insert(y, 0, 0)
  unique = np.insert(unique, 0, unique[0]-1)
  return(unique,y)

# generate simulated data using a histogram data as input
def clone_hist1(data, Nclone):
  x,y = ecdf(data)
  s = np.random.uniform(0,1,Nclone)
  values = []
  for i in range(0, len(s)):
    try:
      idx = np.where(y == y[y<s[i]][-1])[0][0]
      idx=idx+1
    except:
      idx = 0
    values.append(x[idx])
  return values

# generate simulated data using a ECDF curve (x,y)
def clone_hist2(x, y, Nclone):
  s = np.random.uniform(0,1,Nclone)
  values = []
  for i in range(0, len(s)):
    try:
      idx = np.where(y == y[y<s[i]][-1])[0][0]
      idx=idx+1
    except:
      idx = 0
    values.append(x[idx])
  return values

def clone_hist3(x, y, Nclone):
    s = np.random.uniform(0,1,Nclone)
    values = []
    for i in range(0, len(s)):
        idx = np.where(y == y[y<s[i]][-1])[0][0]
        idx=idx+1
        values.append(x[idx])
    return values

def clone_hist4(x, y, Nclone):
    s = np.random.uniform(0,1,Nclone)
    idx = np.searchsorted(y, s, side='left')
    return x[idx]

In [ ]:
############################################################
#generate ecdf map and save it
############################################################
#import matplotlib.pyplot as plt
#from google.colab import files
import time

saveMap = 1

t0 = time.time()

ecdf_map = []

#Nx = 2048
#Ny = 1024
print(Nx)
print(Ny)
k = 0
for i in range(0,Nx):
  if i % 200 == 0:
    print(i)
  ecdf_map.append([])
  for j in range(0,Ny):    
    ecdf_map[i].append([])
    x,y = ecdf(img_real[:,i,j])
    ecdf_map[i][j].append(x)
    ecdf_map[i][j].append(y)
    k = k+1

if saveMap == 1:
  string = str(runI[0])+'_ecdf_map_test'
  np.save(string, np.array(ecdf_map))
  #from google.colab import drive
  #drive.mount('/content/gdrive')
  #path = '/content/gdrive/My Drive/MyNotebooks/Cygno/EcdfPedestal'
  #string = str(runI[0])+'_ecdf_map.npy'
  #!cp  {string} path
  #print(string+' copied to '+path)

t1 = time.time()-t0  
print(t1)

print((t1*2048*2048)/(Nx*Ny))

# THE CODE BELOW IS ONLY FOR TEST

In [ ]:
################################
#test of the ecdf_map variable
################################
plt.rcParams.update({'font.size': 16})

#choose a pixel
i = 1
j = 20

#plot the ECDF of the chosen pixel
plt.step(ecdf_map[i][j][0],ecdf_map[i][j][1], '-',where = 'post')
#plt.plot(ecdf_map1[i][j][0],ecdf_map[i][j][1], ':o') 

plt.grid()

#plot intensities of a pixel from the real image
plt.figure()
plt.plot(img_real[:,i,j])
plt.grid()

print(str(runI[0])+'_ecdf_map')

In [ ]:
############################################################
# generate simulated signal from loaded ecdf_map variable 
############################################################
import time

t0 = time.time()

Nclone = 100000 # a lot of events to have more statistics
Nx = np.size(ecdf_map, 0)
Ny = np.size(ecdf_map, 1)
Nx = 50
Ny = 50
img_sim = np.zeros((Nclone, Nx, Ny), dtype=np.int16)

for i in range(0, Nx):
  if i % 200 == 0:
    print(i)
  for j in range(0, Ny):
    x = ecdf_map[i][j][0]
    y = ecdf_map[i][j][1]    
    img_sim[:,i,j] = clone_hist4(x, y, Nclone)
    
t1 = time.time()-t0  
print(t1)

In [ ]:
############################################################
# compare simulated and real data 
############################################################
plt.rcParams.update({'font.size': 16})

#choose a pixel
i = 1
j = 20

#plot the ECDF of the chosen pixel
plt.step(ecdf_map[i][j][0], ecdf_map[i][j][1], linewidth = 3, where = 'post', label = 'real data')
x,y = ecdf(img_sim[:,i,j])
plt.step(x, y, linestyle = ':', linewidth = 3, where = 'post', label = 'sim. data', color = 'orange')
#plt.plot(ecdf_map1[i][j][0],ecdf_map[i][j][1], ':o') 
plt.legend()
plt.grid()

#plot intensities of a pixel from the real image
plt.figure(figsize=(13, 4))
plt.subplot(121)
plt.plot(img_real[:,i,j], label = 'real data')
plt.grid()
plt.legend()
plt.subplot(122)
plt.plot(img_sim[:,i,j], label = 'sim. data', color = 'orange')
plt.xlim(0,500)
plt.grid()
plt.legend()

#plot histo of intensities of a pixel from the real image
plt.figure()
plt.hist(img_real[:,i,j], np.max(img_real[:,i,j])-np.min(img_real[:,i,j]), linewidth = 3, histtype='step', align='mid', density = 1, label = 'real data')
plt.hist(img_sim[:,i,j], np.max(img_sim[:,i,j])-np.min(img_sim[:,i,j]), linestyle = ':', linewidth = 3, histtype='step', align='mid', density = 1, label = 'sim. data', color = 'orange')
plt.grid()
plt.legend()
